# Lesson 5: Distributed Scaling

In lesson 4 you've learned how to be as efficient as possible on a single CPU.

Now we want to use multiple CPUs at the same time (horizontal scaling) and optimize efficiency of multiple CPUs.

![image](https://raw.githubusercontent.com/nsmith-/2025-09-15-hsf-india-tutorial-chandigarh/cda03896f0f7d4d195db3fedf00a8754f6774214/img/horizontal-and-vertical-scaling.svg)

## Scaling on a single machine

A single computer may have multiple cores available that we can use to concurrently run software.

### In python the two ways to parallelize programs are: **_multi-processing_** and **_multi-threading_**.

**Multi-processing** is essentially about running multiple OS processes in parallel, each with its own Python interpreter, memory space, etc.

**Multi-threading**, however, uses a shared memory space and a common Python interpreter to spawn multiple threads that can run tasks concurrently. 
_Caution_: In Python multi-threading is in fact not running programs in parallel because of the Global Interpreter Lock (GIL), only since python 3.14 (free-threaded) this is possible.


### Basic rule of thumb for Python:

**Use multi-processing** when you're program is CPU-bound (i.e. it's constantly calculating things).

**Use multi-threading** when you're program is IO-bound (i.e. it's waiting a long time for loading data or network requests).

In [1]:
import concurrent.futures

In [2]:
%%writefile quadratic_formula_scaling.py

# writing to a separate file, because multiprocessing is buggy in notebooks (see: https://bugs.python.org/issue25053)
import numpy as np
import time

a = np.random.uniform(5, 10, 1_000_000)
b = np.random.uniform(10, 20, 1_000_000)
c = np.random.uniform(-0.1, 0.1, 1_000_000)

def quadratic_formula(task_id):
    time.sleep(task_id / 5.0)  # simulate some task_id-dependent workload
    return (-b + np.sqrt(b**2 - 4*a*c)), task_id  # return task_id for identification


# setup 10 tasks
task_ids = range(10)

Overwriting quadratic_formula_scaling.py


### Python for-loop example

In [3]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids

for task_id in task_ids:
    output, _ = quadratic_formula(task_id)
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 1 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 2 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 3 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 4 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 5 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 6 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 7 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 8 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 9 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
9.23 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Multi-threading example

In [4]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(quadratic_formula, task_id) for task_id in task_ids]

for future in concurrent.futures.as_completed(futures):
    output, task_id = future.result()
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 9 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 1 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 6 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 3 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 4 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 2 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 0 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 5 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 8 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
Task 7 completed with output [ 0.02510003 -0.01087077 -0.06754239 -0.01035535 -0.00802087]...
1.82 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Multi-processing example

In [5]:
%%timeit -n 1 -r 1

from quadratic_formula_scaling import quadratic_formula, task_ids

with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(quadratic_formula, task_id) for task_id in task_ids]

for future in concurrent.futures.as_completed(futures):
    output, task_id = future.result()
    print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 9 completed with output [-0.06829994  0.07167676 -0.0673028   0.05527705  0.01383229]...
Task 4 completed with output [ 0.06903885 -0.05785964 -0.06256414 -0.05295991 -0.06410584]...
Task 1 completed with output [ 0.07738124 -0.07204735 -0.05872278  0.04306982  0.04349567]...
Task 6 completed with output [-0.06242334 -0.06120943 -0.05140349 -0.05670157 -0.01172167]...
Task 3 completed with output [0.04328633 0.06824904 0.00485421 0.0207105  0.09649188]...
Task 0 completed with output [ 0.02005246  0.04287233  0.04819424 -0.09641227  0.07992221]...
Task 5 completed with output [-0.14977822  0.07326036  0.01172646 -0.13436434  0.00485673]...
Task 7 completed with output [ 0.08055189  0.06281126 -0.08813215 -0.02822091 -0.0009741 ]...
Task 2 completed with output [-0.03501558  0.03862819  0.00268229 -0.050624   -0.00725958]...
Task 8 completed with output [ 0.0280472   0.01394893 -0.01324608 -0.0755722   0.05121581]...
2.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Multi-threading and multi-processing can speed-up your programs by concurrently executing multiple tasks across multiple threads or processes.

Here, we can see in addition the effect of shared (**multi-threading**) vs individual (**multi-processing**) memory space: 
- **multi-threading**: the NumPy random initialization runs _once_ and the memory is shared between all threads, which is why all outputs are the _same_ (`[ 0.00466771  0.00414385  0.00415603 -0.00520032 -0.00212164]...`)
- **multi-processing**: the NumPy random initialization runs in _each_ process again. There's no shared memory, every process has it's own set of `a`, `b`, and `c` arrays, which is why all outputs are _different_. Initializing the arrays in each process adds an additional runtime overhead, which is why here the multi-processing case is slightly slower.

## Scaling on multiple machines

## Dask

There are different ways to scale to multiple machines in a computing cluster. 

The most classic ways are independent of Python and let you submit any type of program 'job' to a cluster, e.g. using HTCondor or Slurm.

In Python, one solution emerged that allows to submit jobs/tasks to a cluster of workers from a Python program: **Dask**.

### Dask example

In [6]:
import dask.distributed

from quadratic_formula_scaling import quadratic_formula, task_ids

In [7]:
%%timeit -n 1 -r 1

with (
    # multi-threading cluster
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # multi-processing cluster
    # dask.distributed.LocalCluster(n_workers=10, threads_per_worker=1) as cluster,
    # mixed cluster
    # dask.distributed.LocalCluster(n_workers=2, threads_per_worker=5) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):

     futures = client.map(quadratic_formula, task_ids)

     for future in dask.distributed.as_completed(futures):
         output, task_id = future.result()
         print(f"Task {task_id} completed with output {output[:5]}...")  # print first 5 results of each task

Task 0 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 1 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 2 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 3 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 4 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 5 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 6 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 7 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 8 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
Task 9 completed with output [ 0.07439047 -0.07153027  0.10357189 -0.04480097  0.06199043]...
2.52 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop ea

### Dask in practice

Dask abstracts the cluster from the user. If there's a dask-scheduler running somewhere, we can connect to it using:

```python
with dask.distributed.Client(address='127.0.0.1:8786') as client: # some IP+port address
    ...
```

This is great, as scientist never have to worry about the cluster infrastructure, with a single lince of code you can connect and run your analysis distributed on multiple workers and optionally using multiple threads per worker.

One cluster in HEP that's commonly used for analysis is: https://coffea.casa/.

### Dask is much more than a distributive interface

Dask allows to define compute graphs of complex workflows. These graphs can then be scheduled automatically by the dask-scheduler such that the most optimal execution of tasks is achieved.

In [8]:
import dask

In [9]:
@dask.delayed  # mark function as delayed/task
def increment(i):
    return i + 1

@dask.delayed  # mark function as delayed/task
def add(a, b):
    return a + b


# define compute graph
a, b = 1, 12
c = increment(a)
d = increment(b)
output = add(c, d)

# visualize compute graph
try:
    output.visualize(rankdir="LR", filename="compute_graph_simple.svg")  # left to right
except Exception as e:
    ... # install graphviz and python-graphviz to visualize the graph

# compute the result
print("Result:", output.compute())

Result: 15


![image](compute_graph_simple.svg)

In [10]:
import numpy as np
import dask.array as da

# use dask arrays instead of Numpy arrays for automatic graph construction
a = da.random.uniform(5, 10, 100_000)
b = da.random.uniform(10, 20, 100_000)
c = da.random.uniform(-0.1, 0.1, 100_000)

def quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

# define compute graph
output = quadratic_formula(a, b, c)

# visualize compute graph
try:
    output.visualize(rankdir="LR", filename="compute_graph_quadratic_formula.svg")
    output.visualize(rankdir="LR", filename="compute_graph_quadratic_formula__optimized.svg", optimize_graph=True)
except Exception as e:
    ... # install graphviz and python-graphviz to visualize the graph


# compute the result
unoptimized_result = dask.compute(output, optimize_graph=False)[0]
print("Result (unoptimized):",unoptimized_result)

optimized_result = dask.compute(output, optimize_graph=True)[0]
print("Result (optimized):",optimized_result)

assert np.allclose(unoptimized_result, optimized_result)


Result (unoptimized): [ 0.00318199  0.00304668  0.00398229 ...  0.00246778 -0.00658297
 -0.00562503]
Result (optimized): [ 0.00318199  0.00304668  0.00398229 ...  0.00246778 -0.00658297
 -0.00562503]


#### Compute Graph

![image](compute_graph_quadratic_formula.svg)

#### Compute Graph _optimized_

![image](compute_graph_quadratic_formula__optimized.svg)

In [11]:
def fun(a, b, c):
    return np.mean((-b + np.sqrt(b**2 - 4*a*c)) / (2*a))

In [12]:
%%timeit -n 1 -r 1

a = np.random.uniform(5, 10, 200_000_000)
b = np.random.uniform(10, 20, 200_000_000)
c = np.random.uniform(-0.1, 0.1, 200_000_000)

output = fun(a, b, c)
print("Result:", output)

Result: -9.418986200643155e-06
3.72 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
# A too slow problem for a single machine... (200M points)

# use dask arrays instead of Numpy arrays for automatic graph construction
da_a = da.random.uniform(5, 10, 200_000_000)
da_b = da.random.uniform(10, 20, 200_000_000)
da_c = da.random.uniform(-0.1, 0.1, 200_000_000)

# define compute graph
output = fun(da_a, da_b, da_c)

print(output.dask)

HighLevelGraph with 15 layers.
 0. uniform-0bc514b519a75ba1cd5b0ad0751ca6da
 1. uniform-1d2f3722e42f6bd69e322c59b8a53e31
 2. mul-fcf875b8cb9da1cea4c98eb212d994e5
 3. mul-59a594da2e4056f85ec81173d566a25b
 4. mul-407ed699c1585dd305ebd1f5b63d8222
 5. uniform-b1045ba093b34ecaa775b9fd1a3bb046
 6. pow-d333d690f3c74d88b3ce0c32bbb2a063
 7. sub-edb58060ef6c850ade51ea00fc3162e0
 8. sqrt-31d7a3939c1d1a01095c91dfae9d0c9d
 9. neg-77668f83c8a77cf82b10b5f26f67d5f8
 10. add-c61b87af3d16facf8590ef76b1d699ee
 11. truediv-525df2aefae68852310552d51c8c2081
 12. mean_chunk-06f64997f99bfa0d0df8a8041f25716c
 13. mean_combine-partial-9bc30f2506ad9bdfb248d2a22dd98272
 14. mean_agg-aggregate-978b5aeead1e0f863d5175e3daff813f



In [14]:
%%timeit -n 1 -r 1

with (
    dask.distributed.LocalCluster(n_workers=1, threads_per_worker=10) as cluster,
    # ---
    # connect to a Dask cluster
    dask.distributed.Client(cluster) as client,
):
    # compute the result
    print("Result:", client.compute(output).result())

Result: -9.050460438113965e-06
1.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
